## Investigation about duplicate InChI-Keys in the 'ML-ready' DFs

### Problem Statement
In the notebook 'pubchem-sparse-matrix' on the Jovian Server it became evident that the Metadata_broad_sample identifier of the cp_annotations.csv file is not suited as a unique identifier. The DF resulting from overlapping on the Metadata_broad_sample has duplicate inchikeys. The notebook 'Troubleshooting' in this directory illuminated that the Identifier columns CPD_SMILES, inchikey, contained duplicate values.

### Aim of this Notebook
In this notebook the cause of duplicate InChI-Keys is to be investigated. For this purpose the columns 'inchikey', 'CPD_SMILES' and 'Metadata_broad_sample' (MBS) are taken from the file 'cp_annotations.csv' and the non-duplicate InChI-Keys are being dropped. The DF is then sorted by InChI-Keys to reveal the competing Metadata_broad_sample-Identifier (MBS). The resulting DF is then merged with the file 'raw_image_data.txt' on the column 'Metadata_broad_sample' (MBS). From the resulting dataframe all CellProfiler Columns and invariant Metadata columns are being dropped. The DF is then inspected visually to find patterns relating InChI-Keys and Metadata_broad_sample (MBS) to other information available in the Dataframe. Finally Hypothesis are formulated which are then tested against the given data. In this case the hypothesis is that the MBS of a given InChI-Key matches either the 'Metadata_Plate_Map_Number'-Column (MPMN) and the 'Metadata_mmoles_per_liter'-Columns (CONC) or just the MPMN or just the CONC which was found to be true.

### Summary of the Steps Undertaken
1. Get Relevant Identifiers from Annotations File
2. Get Relevant Rows in sorted order
3. Merge with Raw Data
4. Inspect Metadata Columns to find Patterns
5. Find precise relation of MBS to other Metadata

### Conclusion
Different MBS that are related to the same InChI-Key have either the same MPMN and the same CONC or just the same MPMN or just the same CONC. This was proven in step 5. In the corresponding jupyter-cell (s.b.) three conditional statements compared the number of unique MBS of one InChI-key with the nunmber of unique MPMN and unique CONC aswell as just with the number of unique MPMN aswell as just the number of unique CONC. Should these numbers be the same subsub-dataframes were built from the unique values of said columns and their MBS-list was compared. The same MBS-list in each of the subsub-dfs means that the MBS correspond the the respective column (either MPMN, CONC or both) which was the case for all 400 unique MBS that had duplicate InChI-keys.
That means, that the Preprocessing of the 'raw_image_data.txt' has to be revisited. More rows can be combined because of same concentration ranges and so on.
It can also be seen that Compounds with different CONC are fairly close together and might be able to be combined into one average value.

In [71]:
# Prerequisite

import pandas as pd
df = pd.read_csv("../../input/cp_annotations.csv")

In [72]:
# 1. Get Relevant Identifiers from Annotations File
df = df.loc[:,['inchikey','CPD_SMILES','Metadata_broad_sample']]
df['duplicate'] = df['inchikey'].duplicated(keep=False)
df['duplicateSMILES'] = df['CPD_SMILES'].duplicated(keep=False)

In [73]:
# 2. Get Relevant Rows in sorted order
df = df.query('duplicate == True')
df = df.dropna(subset=['inchikey'])
df = df.sort_values(by=['inchikey'])

In [74]:
# non essential control step
smiles_twice = ((df.CPD_SMILES.value_counts() > 2).to_numpy().sum() == 0) & ((df.CPD_SMILES.value_counts() < 2).to_numpy().sum() == 0)
inchikey_twice = ((df.inchikey.value_counts() > 2).to_numpy().sum() == 0) & ((df.inchikey.value_counts() < 2).to_numpy().sum() == 0)
unique_smiles = df.CPD_SMILES.value_counts().shape[0]
unique_inchikey = df.inchikey.value_counts().shape[0]

print('SMILES only twice? {}!'.format(smiles_twice))
print('InChIkeys only twice? {}!'.format(inchikey_twice))
print('#uniqueSMILES: {}'.format(unique_smiles))
print('#uniqueInChIKey: {}'.format(unique_inchikey))

SMILES only twice? False!
InChIkeys only twice? True!
#uniqueSMILES: 202
#uniqueInChIKey: 200


In [75]:
# 3. Merge with Raw Data
meta_data_cols = ['Metadata_broad_sample', 'Metadata_Plate','Metadata_Plate_Map_Name',
                  'Metadata_mmoles_per_liter', 'Metadata_pert_id','Metadata_pert_mfc_id', 'Metadata_pert_well']

df = pd.merge(left=df, right=pd.read_csv("../../input/Raw_Image_data.txt",usecols=meta_data_cols, sep='\t'), how='outer', on='Metadata_broad_sample')
df = df.dropna(subset=['duplicate'])

In [76]:
# 4. Inspect Metadata Columns to find Patterns

pd.set_option('max_colwidth', 25)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df[['inchikey','Metadata_broad_sample', 'Metadata_Plate_Map_Name']])

                      inchikey   Metadata_broad_sample Metadata_Plate_Map_Name
0     APDMZOWNVCUUKT-LQWHRV...  BRD-K40294449-001-02-1            H-CBLD-002-4
1     APDMZOWNVCUUKT-LQWHRV...  BRD-K40294449-001-02-1            H-CBLD-002-4
2     APDMZOWNVCUUKT-LQWHRV...  BRD-K40294449-001-02-1            H-CBLD-002-4
3     APDMZOWNVCUUKT-LQWHRV...  BRD-K40294449-001-02-1            H-CBLD-002-4
4     APDMZOWNVCUUKT-LQWHRV...  BRD-K40294449-001-01-3            H-CBLD-004-4
5     APDMZOWNVCUUKT-LQWHRV...  BRD-K40294449-001-01-3            H-CBLD-004-4
6     APDMZOWNVCUUKT-LQWHRV...  BRD-K40294449-001-01-3            H-CBLD-004-4
7     APDMZOWNVCUUKT-LQWHRV...  BRD-K40294449-001-01-3            H-CBLD-004-4
8     APDMZOWNVCUUKT-OPYAII...  BRD-K76703685-001-01-3            H-CBLD-004-4
9     APDMZOWNVCUUKT-OPYAII...  BRD-K76703685-001-01-3            H-CBLD-004-4
10    APDMZOWNVCUUKT-OPYAII...  BRD-K76703685-001-01-3            H-CBLD-004-4
11    APDMZOWNVCUUKT-OPYAII...  BRD-K76703685-001-01

In [78]:
# 5. Find precise relation of MBS to other Metadata

# liste mit inchie keys machen
unique_inchikeys = df.inchikey.drop_duplicates().to_list()

# variables for counting the MBS that match MPMN and CONC respectively
MBS_matching_MPMN = 0
MBS_matching_CONC = 0
MBS_matching_MPMN_CONC = 0

#iterate through all unique inchies
for inchi in unique_inchikeys:
    # slice the df into a sub-df corresponding to the first unique inchikey
    sub_df_inchikey = df.query('inchikey == @inchi')
    
    # make lists of unique MBS MPMN and CONC present in the sub_df
    unique_MBS = sub_df_inchikey.Metadata_broad_sample.drop_duplicates().to_list()
    unique_MPMN = sub_df_inchikey.Metadata_Plate_Map_Name.drop_duplicates().to_list()
    unique_CONC = sub_df_inchikey.Metadata_mmoles_per_liter.drop_duplicates().to_list()
    
    # do the number of unique MBS match the number of unique MPMN entries for this sub_df?
    if (len(unique_MBS) == len(unique_MPMN)) & (len(unique_MBS) == len(unique_CONC)):
        # cycle through the number of unique MBS entries (== number of unique MPMN & CONC entries)
        for i in range(len(unique_MBS)):
            
            # creat subsub_dfs corresponding to either the MBS or MPMN or CONC
            subsub_df_MBS = sub_df_inchikey.query('Metadata_broad_sample == @unique_MBS[@i]')
            subsub_df_MPMN = sub_df_inchikey.query('Metadata_Plate_Map_Name == @unique_MPMN[@i]')
            subsub_df_CONC = sub_df_inchikey.query('Metadata_mmoles_per_liter == @unique_CONC[@i]')
            
            # are the subsub_dfs identical to each other ?
            if (subsub_df_MPMN.Metadata_broad_sample.to_list() == subsub_df_MBS.Metadata_broad_sample.to_list() == subsub_df_CONC.Metadata_broad_sample.to_list()):
                # if so the given MBS matches with the MPMN
                MBS_matching_MPMN_CONC += 1
                ####### print statement just to see how close different concentrations are
                print(sub_df_inchikey)
            # are the subsub_dfs different from each other ?
            else:
                # that means MBS does not correlate with MPMN and CONC even though their numbers match
                print("#MBS = #CONC but the subsubdfs are not identical!\n", sub_df_inchikey)
    
    # do the number of unique MBS match the number of unique MPMN entries for this sub_df?
    elif len(unique_MBS) == len(unique_MPMN):
        # cycle through the number of unique MBS entries (== number of unique MPMN entries)
        for i in range(len(unique_MBS)):
            
            # creat subsub_dfs corresponding to either the MBS or MPMN
            subsub_df_MBS = sub_df_inchikey.query('Metadata_broad_sample == @unique_MBS[@i]')
            subsub_df_MPMN = sub_df_inchikey.query('Metadata_Plate_Map_Name == @unique_MPMN[@i]')
            
            # are the subsub_dfs identical to each other ?
            if (subsub_df_MPMN.Metadata_broad_sample.to_list() == subsub_df_MBS.Metadata_broad_sample.to_list()):
                # if so the given MBS matches with the MPMN
                MBS_matching_MPMN += 1
            # are the subsub_dfs different from each other ?
            else:
                # that means MBS does not correlate with MPMN even though their numbers match
                print("#MBS = #CONC but the subsubdfs are not identical!\n", sub_df_inchikey)
    
    # do the number of unique MBS match the number of unique CONC entries for this sub_df?
    elif len(unique_MBS) == len(unique_CONC):
        # cycle through the number of unique MBS entries (== number of unique CONC entries)
        for i in range(len(unique_MBS)):
            
            # creat subsub_dfs corresponding to either the MBS or CONC
            subsub_df_MBS = sub_df_inchikey.query('Metadata_broad_sample == @unique_MBS[@i]')
            subsub_df_CONC = sub_df_inchikey.query('Metadata_mmoles_per_liter == @unique_CONC[@i]')
            
            # are the subsub_dfs identical to each other ?
            if (subsub_df_MBS.Metadata_broad_sample.to_list() == subsub_df_CONC.Metadata_broad_sample.to_list()):
                #if so the given MBS matches with the CONC
                MBS_matching_CONC += 1
            # are the subsub_dfs different from each other ?
            else:
                # that means MBS does not correlate with MPMN even though their numbers match
                print("#MBS = #CONC but the subsubdfs are not identical!\n", sub_df_inchikey)
    else:
        print("# of unique MBS does match # of either MPMN or CONC\n", inchi)

print('MBS matching MPMN & CONC: {}'.format(MBS_matching_MPMN_CONC))
print('MBS matching MPMN only:   {}'.format(MBS_matching_MPMN))
print('MBS matching CONC only:   {}'.format(MBS_matching_CONC))
print('MBS matching either MPMN & CONC or (CONC or MPMN): {}'.format(MBS_matching_CONC+MBS_matching_MPMN+MBS_matching_MPMN_CONC))
print('total unique MBS:  {}'.format(df.Metadata_broad_sample.drop_duplicates().shape[0]))

                   inchikey                CPD_SMILES   Metadata_broad_sample  \
0  APDMZOWNVCUUKT-LQWHRV...  C[C@@H](CO)N1C[C@@H](...  BRD-K40294449-001-02-1   
1  APDMZOWNVCUUKT-LQWHRV...  C[C@@H](CO)N1C[C@@H](...  BRD-K40294449-001-02-1   
2  APDMZOWNVCUUKT-LQWHRV...  C[C@@H](CO)N1C[C@@H](...  BRD-K40294449-001-02-1   
3  APDMZOWNVCUUKT-LQWHRV...  C[C@@H](CO)N1C[C@@H](...  BRD-K40294449-001-02-1   
4  APDMZOWNVCUUKT-LQWHRV...  C[C@@H](CO)N1C[C@@H](...  BRD-K40294449-001-01-3   
5  APDMZOWNVCUUKT-LQWHRV...  C[C@@H](CO)N1C[C@@H](...  BRD-K40294449-001-01-3   
6  APDMZOWNVCUUKT-LQWHRV...  C[C@@H](CO)N1C[C@@H](...  BRD-K40294449-001-01-3   
7  APDMZOWNVCUUKT-LQWHRV...  C[C@@H](CO)N1C[C@@H](...  BRD-K40294449-001-01-3   

  duplicate duplicateSMILES  Metadata_Plate Metadata_Plate_Map_Name  \
0      True            True           24734            H-CBLD-002-4   
1      True            True           24735            H-CBLD-002-4   
2      True            True           24736            H-

                    inchikey                CPD_SMILES  \
64  BSKZHQCAOYVBFU-RELGLD...  CN1[C@@H]2CC[C@@H](CC...   
65  BSKZHQCAOYVBFU-RELGLD...  CN1[C@@H]2CC[C@@H](CC...   
66  BSKZHQCAOYVBFU-RELGLD...  CN1[C@@H]2CC[C@@H](CC...   
67  BSKZHQCAOYVBFU-RELGLD...  CN1[C@@H]2CC[C@@H](CC...   
68  BSKZHQCAOYVBFU-RELGLD...  CN1[C@@H]2CC[C@@H](CC...   
69  BSKZHQCAOYVBFU-RELGLD...  CN1[C@@H]2CC[C@@H](CC...   
70  BSKZHQCAOYVBFU-RELGLD...  CN1[C@@H]2CC[C@@H](CC...   
71  BSKZHQCAOYVBFU-RELGLD...  CN1[C@@H]2CC[C@@H](CC...   

     Metadata_broad_sample duplicate duplicateSMILES  Metadata_Plate  \
64  BRD-K06039860-001-01-4      True            True           26640   
65  BRD-K06039860-001-01-4      True            True           26641   
66  BRD-K06039860-001-01-4      True            True           26642   
67  BRD-K06039860-001-01-4      True            True           26643   
68  BRD-K06039860-001-02-2      True            True           26767   
69  BRD-K06039860-001-02-2      True         

                     inchikey                CPD_SMILES  \
111  BWTMYHAPSQQINJ-IECBHU...  C[C@H](CO)N1C[C@@H](C...   
112  BWTMYHAPSQQINJ-IECBHU...  C[C@H](CO)N1C[C@@H](C...   
113  BWTMYHAPSQQINJ-IECBHU...  C[C@H](CO)N1C[C@@H](C...   
114  BWTMYHAPSQQINJ-IECBHU...  C[C@H](CO)N1C[C@@H](C...   
115  BWTMYHAPSQQINJ-IECBHU...  C[C@H](CO)N1C[C@@H](C...   
116  BWTMYHAPSQQINJ-IECBHU...  C[C@H](CO)N1C[C@@H](C...   
117  BWTMYHAPSQQINJ-IECBHU...  C[C@H](CO)N1C[C@@H](C...   
118  BWTMYHAPSQQINJ-IECBHU...  C[C@H](CO)N1C[C@@H](C...   

      Metadata_broad_sample duplicate duplicateSMILES  Metadata_Plate  \
111  BRD-K47380281-001-02-2      True            True           24726   
112  BRD-K47380281-001-02-2      True            True           24731   
113  BRD-K47380281-001-02-2      True            True           24732   
114  BRD-K47380281-001-02-2      True            True           24733   
115  BRD-K47380281-001-01-4      True            True           26679   
116  BRD-K47380281-001-01-4   

                     inchikey                CPD_SMILES  \
143  CMMUKIKJYLMGMW-YQQAZP...  C[C@@H](CO)N1C[C@H](C...   
144  CMMUKIKJYLMGMW-YQQAZP...  C[C@@H](CO)N1C[C@H](C...   
145  CMMUKIKJYLMGMW-YQQAZP...  C[C@@H](CO)N1C[C@H](C...   
146  CMMUKIKJYLMGMW-YQQAZP...  C[C@@H](CO)N1C[C@H](C...   
147  CMMUKIKJYLMGMW-YQQAZP...  C[C@@H](CO)N1C[C@H](C...   
148  CMMUKIKJYLMGMW-YQQAZP...  C[C@@H](CO)N1C[C@H](C...   
149  CMMUKIKJYLMGMW-YQQAZP...  C[C@@H](CO)N1C[C@H](C...   
150  CMMUKIKJYLMGMW-YQQAZP...  C[C@@H](CO)N1C[C@H](C...   

      Metadata_broad_sample duplicate duplicateSMILES  Metadata_Plate  \
143  BRD-K57882732-001-01-1      True            True           26679   
144  BRD-K57882732-001-01-1      True            True           26680   
145  BRD-K57882732-001-01-1      True            True           26794   
146  BRD-K57882732-001-01-1      True            True           26795   
147  BRD-K57882732-001-02-9      True            True           24734   
148  BRD-K57882732-001-02-9   

                     inchikey                CPD_SMILES  \
187  DEGZSTUNWMGQCY-GSHUGG...  C[C@H](CO)N1C[C@H](C)...   
188  DEGZSTUNWMGQCY-GSHUGG...  C[C@H](CO)N1C[C@H](C)...   
189  DEGZSTUNWMGQCY-GSHUGG...  C[C@H](CO)N1C[C@H](C)...   
190  DEGZSTUNWMGQCY-GSHUGG...  C[C@H](CO)N1C[C@H](C)...   
191  DEGZSTUNWMGQCY-GSHUGG...  C[C@H](CO)N1C[C@H](C)...   
192  DEGZSTUNWMGQCY-GSHUGG...  C[C@H](CO)N1C[C@H](C)...   
193  DEGZSTUNWMGQCY-GSHUGG...  C[C@H](CO)N1C[C@H](C)...   
194  DEGZSTUNWMGQCY-GSHUGG...  C[C@H](CO)N1C[C@H](C)...   

      Metadata_broad_sample duplicate duplicateSMILES  Metadata_Plate  \
187  BRD-K19471718-001-02-6      True            True           24726   
188  BRD-K19471718-001-02-6      True            True           24731   
189  BRD-K19471718-001-02-6      True            True           24732   
190  BRD-K19471718-001-02-6      True            True           24733   
191  BRD-K19471718-001-01-8      True            True           26679   
192  BRD-K19471718-001-01-8   

...

                      inchikey                CPD_SMILES  \
1422  WMEBRZUCENJNIY-UEXGIB...  C[C@H](CO)N1C[C@H](C)...   
1423  WMEBRZUCENJNIY-UEXGIB...  C[C@H](CO)N1C[C@H](C)...   
1424  WMEBRZUCENJNIY-UEXGIB...  C[C@H](CO)N1C[C@H](C)...   
1425  WMEBRZUCENJNIY-UEXGIB...  C[C@H](CO)N1C[C@H](C)...   
1426  WMEBRZUCENJNIY-UEXGIB...  C[C@H](CO)N1C[C@H](C)...   
1427  WMEBRZUCENJNIY-UEXGIB...  C[C@H](CO)N1C[C@H](C)...   
1428  WMEBRZUCENJNIY-UEXGIB...  C[C@H](CO)N1C[C@H](C)...   

       Metadata_broad_sample duplicate duplicateSMILES  Metadata_Plate  \
1422  BRD-K27248268-001-02-2      True            True           24740   
1423  BRD-K27248268-001-02-2      True            True           24750   
1424  BRD-K27248268-001-02-2      True            True           24751   
1425  BRD-K27248268-001-01-4      True            True           26679   
1426  BRD-K27248268-001-01-4      True            True           26680   
1427  BRD-K27248268-001-01-4      True            True           26794   
1

                      inchikey                CPD_SMILES  \
1459  WYPGSQIBOYUSAK-CZSZKK...  C[C@@H](CO)N1C[C@@H](...   
1460  WYPGSQIBOYUSAK-CZSZKK...  C[C@@H](CO)N1C[C@@H](...   
1461  WYPGSQIBOYUSAK-CZSZKK...  C[C@@H](CO)N1C[C@@H](...   
1462  WYPGSQIBOYUSAK-CZSZKK...  C[C@@H](CO)N1C[C@@H](...   
1463  WYPGSQIBOYUSAK-CZSZKK...  C[C@@H](CO)N1C[C@@H](...   
1464  WYPGSQIBOYUSAK-CZSZKK...  C[C@@H](CO)N1C[C@@H](...   
1465  WYPGSQIBOYUSAK-CZSZKK...  C[C@@H](CO)N1C[C@@H](...   
1466  WYPGSQIBOYUSAK-CZSZKK...  C[C@@H](CO)N1C[C@@H](...   

       Metadata_broad_sample duplicate duplicateSMILES  Metadata_Plate  \
1459  BRD-K04749191-001-01-0      True            True           26679   
1460  BRD-K04749191-001-01-0      True            True           26680   
1461  BRD-K04749191-001-01-0      True            True           26794   
1462  BRD-K04749191-001-01-0      True            True           26795   
1463  BRD-K04749191-001-02-8      True            True           24726   
1464  BRD-K0474

                      inchikey                CPD_SMILES  \
1506  XYWPMUCHOREPKK-AXHZCL...  C[C@@H](CO)N1C[C@H](C...   
1507  XYWPMUCHOREPKK-AXHZCL...  C[C@@H](CO)N1C[C@H](C...   
1508  XYWPMUCHOREPKK-AXHZCL...  C[C@@H](CO)N1C[C@H](C...   
1509  XYWPMUCHOREPKK-AXHZCL...  C[C@@H](CO)N1C[C@H](C...   
1510  XYWPMUCHOREPKK-AXHZCL...  C[C@@H](CO)N1C[C@H](C...   
1511  XYWPMUCHOREPKK-AXHZCL...  C[C@@H](CO)N1C[C@H](C...   
1512  XYWPMUCHOREPKK-AXHZCL...  C[C@@H](CO)N1C[C@H](C...   
1513  XYWPMUCHOREPKK-AXHZCL...  C[C@@H](CO)N1C[C@H](C...   

       Metadata_broad_sample duplicate duplicateSMILES  Metadata_Plate  \
1506  BRD-K60129749-001-02-1      True            True           24726   
1507  BRD-K60129749-001-02-1      True            True           24731   
1508  BRD-K60129749-001-02-1      True            True           24732   
1509  BRD-K60129749-001-02-1      True            True           24733   
1510  BRD-K60129749-001-01-3      True            True           26679   
1511  BRD-K6012

                      inchikey                CPD_SMILES  \
1550  YRBTUIBMTXMQGX-DXIQSL...  C[C@@H](CO)N1C[C@@H](...   
1551  YRBTUIBMTXMQGX-DXIQSL...  C[C@@H](CO)N1C[C@@H](...   
1552  YRBTUIBMTXMQGX-DXIQSL...  C[C@@H](CO)N1C[C@@H](...   
1553  YRBTUIBMTXMQGX-DXIQSL...  C[C@@H](CO)N1C[C@@H](...   
1554  YRBTUIBMTXMQGX-DXIQSL...  C[C@@H](CO)N1C[C@@H](...   
1555  YRBTUIBMTXMQGX-DXIQSL...  C[C@@H](CO)N1C[C@@H](...   
1556  YRBTUIBMTXMQGX-DXIQSL...  C[C@@H](CO)N1C[C@@H](...   
1557  YRBTUIBMTXMQGX-DXIQSL...  C[C@@H](CO)N1C[C@@H](...   

       Metadata_broad_sample duplicate duplicateSMILES  Metadata_Plate  \
1550  BRD-K86033856-001-01-6      True            True           26679   
1551  BRD-K86033856-001-01-6      True            True           26680   
1552  BRD-K86033856-001-01-6      True            True           26794   
1553  BRD-K86033856-001-01-6      True            True           26795   
1554  BRD-K86033856-001-02-4      True            True           24734   
1555  BRD-K8603

                      inchikey                CPD_SMILES  \
1588  ZMFVFBQYBHAIKE-VCBZYW...  C[C@@H](CO)N1C[C@@H](...   
1589  ZMFVFBQYBHAIKE-VCBZYW...  C[C@@H](CO)N1C[C@@H](...   
1590  ZMFVFBQYBHAIKE-VCBZYW...  C[C@@H](CO)N1C[C@@H](...   
1591  ZMFVFBQYBHAIKE-VCBZYW...  C[C@@H](CO)N1C[C@@H](...   
1592  ZMFVFBQYBHAIKE-VCBZYW...  C[C@@H](CO)N1C[C@@H](...   
1593  ZMFVFBQYBHAIKE-VCBZYW...  C[C@@H](CO)N1C[C@@H](...   
1594  ZMFVFBQYBHAIKE-VCBZYW...  C[C@@H](CO)N1C[C@@H](...   
1595  ZMFVFBQYBHAIKE-VCBZYW...  C[C@@H](CO)N1C[C@@H](...   

       Metadata_broad_sample duplicate duplicateSMILES  Metadata_Plate  \
1588  BRD-K60520051-001-02-3      True            True           24726   
1589  BRD-K60520051-001-02-3      True            True           24731   
1590  BRD-K60520051-001-02-3      True            True           24732   
1591  BRD-K60520051-001-02-3      True            True           24733   
1592  BRD-K60520051-001-01-5      True            True           26679   
1593  BRD-K6052